##Downloading the Dataset from the kaggle


In [ ]:
import kagglehub
import os

# Download the dataset
dataset_path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Dataset downloaded to:", dataset_path)

Using Colab cache for faster access to the 'gtsrb-german-traffic-sign' dataset.
Dataset downloaded to: /kaggle/input/gtsrb-german-traffic-sign


# dataset Preprocessing
##1: Loading dataset from the tensorflow
##2: Resizing imges to 64x64 for consistency.
##3: Batch_size=32(means that load 32 images at a time)

In [ ]:
import tensorflow as tf
import os

data_dir = os.path.join(dataset_path, "Train")

# Load training set
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # 20% for validation
    subset="training",      # This is the training subset
    seed=123,               # Ensures reproducibility
    image_size=(64, 64),
    batch_size=32
)

# Load validation set
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # Same split as above
    subset="validation",    # This is the validation subset
    seed=123,
    image_size=(64, 64),
    batch_size=32
)


Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Found 39209 files belonging to 43 classes.
Using 7841 files for validation.


In [ ]:
class_names = train_ds.class_names
print("Classes:", class_names)
print("Number of classes:", len(class_names))

# Take one batch to confirm
for images, labels in train_ds.take(1):
    print("Train batch shape:", images.shape)
    print("Train labels:", labels[:5].numpy())


Classes: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '5', '6', '7', '8', '9']
Number of classes: 43
Train batch shape: (32, 64, 64, 3)
Train labels: [ 5 17  9  5  3]


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


# Normalize values to [0,1]


In [ ]:
from tensorflow.keras import layers

normalization_layer = layers.Rescaling(1./255)

# Apply it to your datasets
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
for images, labels in train_ds.take(1):
    print("Min pixel value:", tf.reduce_min(images).numpy())
    print("Max pixel value:", tf.reduce_max(images).numpy())


Min pixel value: 0.016743261
Max pixel value: 1.0


#Data Augmentation

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

# Create a data augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),        # Randomly flips images horizontally
    layers.RandomRotation(0.1),             # Rotates images randomly (±10%)
    layers.RandomZoom(0.1),                 # Randomly zooms in/out
    layers.RandomTranslation(0.1, 0.1),     # Randomly shifts images
])


# Model training
##architecture of this model:
### 3 CNN layers
### 2 Maxpooling2D layers
### dropout 0.5
### hidden layer activation function Relu
### output layer activatio function softmax
### Adam optimizer
### loss function: sparse categorical cross entropy


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Build CNN model
model = models.Sequential([
    # Convolutional Layer 1
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(pool_size=(2,2)),

    # Convolutional Layer 2
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2,2)),

    # Convolutional Layer 3
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),

    # Flatten and Dense layers
    layers.Flatten(),
    layers.Dropout(0.5),                   # 🔹 Dropout for regularization
    layers.Dense(128, activation='relu'),
    layers.Dense(43, activation='softmax') # 🔹 Output layer (43 classes)
])

# Compile the model
model.compile(
    optimizer='adam',                      # Adam optimizer
    loss='sparse_categorical_crossentropy',     # For multi-class classification
    metrics=['accuracy']
)

# Implement Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,                            # Stop after 3 epochs without improvement
    restore_best_weights=True
)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping]
)


Epoch 1/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 339s 344ms/step - accuracy: 0.5863 - loss: 1.5131 - val_accuracy: 0.9666 - val_loss: 0.1349
Epoch 2/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 356s 318ms/step - accuracy: 0.9671 - loss: 0.1156 - val_accuracy: 0.9781 - val_loss: 0.0917
Epoch 3/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 318s 313ms/step - accuracy: 0.9834 - loss: 0.0558 - val_accuracy: 0.9853 - val_loss: 0.0776
Epoch 4/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 318s 310ms/step - accuracy: 0.9903 - loss: 0.0313 - val_accuracy: 0.9842 - val_loss: 0.0714
Epoch 5/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 306s 312ms/step - accuracy: 0.9905 - loss: 0.0278 - val_accuracy: 0.9875 - val_loss: 0.0670
Epoch 6/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 329s 319ms/step - accuracy: 0.9935 - loss: 0.0226 - val_accuracy: 0.9860 - val_loss: 0.0744
Epoch 7/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 310s 316ms/step - accuracy: 0.9932 - loss: 0.0242 - val_accuracy: 0.9912 - val_loss: 0.0460
Epoch 8/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 311s 317ms/step - accuracy: 0.9959 -

#Transfer Learning(VGG16)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix


# Task
Compare the performance of a custom CNN and a VGG16 model with transfer learning for classifying traffic signs into 43 classes. The comparison should include accuracy, loss, confusion matrix, and performance metrics (accuracy, precision, recall, F1-score).

## Load the vgg16 model

### Subtask:
Load the VGG16 model with pre-trained weights, excluding the top classification layer.


**Reasoning**:
Load the VGG16 model with pre-trained weights, excluding the top classification layer, and print its summary.



In [ ]:
from tensorflow.keras.applications import VGG16

# Load the VGG16 model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Print the model summary
vgg_base.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 64, 64, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

## Build the new model

### Subtask:
Add new layers on top of the VGG16 base for the 43-class classification task.


**Reasoning**:
Add new layers on top of the VGG16 base for the 43-class classification task as per the instructions.



In [ ]:
# Define the number of classes
num_classes = 43

# Create the new model
transfer_model = models.Sequential([
    # Add the data augmentation layer
    data_augmentation,

    # Add the pre-trained VGG16 base model
    vgg_base,

    # Flatten the output of the VGG16 base
    layers.Flatten(),

    # Add a Dense hidden layer with ReLU activation
    layers.Dense(128, activation='relu'),

    # Add a Dropout layer for regularization
    layers.Dropout(0.5),

    # Add the final Dense output layer with softmax activation
    layers.Dense(num_classes, activation='softmax')
])

# Freeze the layers of the VGG16 base model
for layer in vgg_base.layers:
    layer.trainable = False

# Print the model summary
transfer_model.summary()

NameError: name 'data_augmentation' is not defined

**Reasoning**:
The variable `data_augmentation` was not defined in the current execution. The error indicates that the code cell where `data_augmentation` was defined (cell_id: hIagvcSpq_K-) needs to be executed first. I will regenerate the code block including the definition of `data_augmentation`.



In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

# Create a data augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),        # Randomly flips images horizontally
    layers.RandomRotation(0.1),             # Rotates images randomly (±10%)
    layers.RandomZoom(0.1),                 # Randomly zooms in/out
    layers.RandomTranslation(0.1, 0.1),     # Randomly shifts images
])


# Define the number of classes
num_classes = 43

# Create the new model
transfer_model = models.Sequential([
    # Add the data augmentation layer
    data_augmentation,

    # Add the pre-trained VGG16 base model
    vgg_base,

    # Flatten the output of the VGG16 base
    layers.Flatten(),

    # Add a Dense hidden layer with ReLU activation
    layers.Dense(128, activation='relu'),

    # Add a Dropout layer for regularization
    layers.Dropout(0.5),

    # Add the final Dense output layer with softmax activation
    layers.Dense(num_classes, activation='softmax')
])

# Freeze the layers of the VGG16 base model
for layer in vgg_base.layers:
    layer.trainable = False

# Print the model summary
transfer_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 2, 2, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

## Compile the model

### Subtask:
Compile the new model with an appropriate optimizer, loss function, and metrics.


**Reasoning**:
Compile the `transfer_model` with the specified optimizer, loss function, and metrics.



In [ ]:
# Compile the transfer model
transfer_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Print the model summary after compilation to verify
transfer_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 2, 2, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

## Train the model (frozen layers)

### Subtask:
Train the new layers while keeping the VGG16 base layers frozen.


**Reasoning**:
Implement early stopping and train the transfer learning model with the frozen VGG16 base.



In [ ]:
# Implement Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,                            # Stop after 3 epochs without improvement
    restore_best_weights=True
)

# Train the model
transfer_history_frozen = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping]
)

NameError: name 'train_ds' is not defined

**Reasoning**:
The previous command failed because the `train_ds` and `val_ds` datasets were not defined in the current session. Re-execute the cells that define and preprocess the datasets.



In [ ]:
import tensorflow as tf
import os

data_dir = os.path.join(dataset_path, "Train")

# Load training set
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # 20% for validation
    subset="training",      # This is the training subset
    seed=123,               # Ensures reproducibility
    image_size=(64, 64),
    batch_size=32
)

# Load validation set
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # Same split as above
    subset="validation",    # This is the validation subset
    seed=123,
    image_size=(64, 64),
    batch_size=32
)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

from tensorflow.keras import layers

normalization_layer = layers.Rescaling(1./255)

# Apply it to your datasets
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Implement Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,                            # Stop after 3 epochs without improvement
    restore_best_weights=True
)

# Train the model
transfer_history_frozen = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping]
)

NameError: name 'dataset_path' is not defined

**Reasoning**:
The variable `dataset_path` is not defined. The cell that downloads the dataset and defines this variable needs to be executed. I will re-execute the cell that downloads the dataset and then re-execute the previous command to load, preprocess, and train the model.



In [9]:
import kagglehub
import os

# Download the dataset
dataset_path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Dataset downloaded to:", dataset_path)

import tensorflow as tf
from tensorflow.keras import layers

data_dir = os.path.join(dataset_path, "Train")

# Load training set
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # 20% for validation
    subset="training",      # This is the training subset
    seed=123,               # Ensures reproducibility
    image_size=(64, 64),
    batch_size=32
)

# Load validation set
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,   # Same split as above
    subset="validation",    # This is the validation subset
    seed=123,
    image_size=(64, 64),
    batch_size=32
)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.Rescaling(1./255)

# Apply it to your datasets
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Implement Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,                            # Stop after 3 epochs without improvement
    restore_best_weights=True
)

# Train the model
transfer_history_frozen = transfer_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping]
)

Using Colab cache for faster access to the 'gtsrb-german-traffic-sign' dataset.
Dataset downloaded to: /kaggle/input/gtsrb-german-traffic-sign
Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Found 39209 files belonging to 43 classes.
Using 7841 files for validation.
Epoch 1/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 2026s 2s/step - accuracy: 0.2535 - loss: 2.6634 - val_accuracy: 0.5100 - val_loss: 1.5730
Epoch 2/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 1960s 2s/step - accuracy: 0.4344 - loss: 1.7885 - val_accuracy: 0.5773 - val_loss: 1.3607
Epoch 3/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 1929s 2s/step - accuracy: 0.4719 - loss: 1.6262 - val_accuracy: 0.6203 - val_loss: 1.2212
Epoch 4/10
981/981 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5029 - loss: 1.5242

KeyboardInterrupt: 